In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from itertools import combinations
import missingno as msno
import matplotlib.ticker as mticker # NameError 해결을 위한 Ticker 임포트
import re
from typing import List, Dict, Union
from pandas.api.types import is_datetime64_any_dtype

try:
    plt.rcParams['font.family'] = 'Malgun Gothic'
except:
    plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False


In [2]:
def downcast(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        dtype_name = df[col].dtype.name
        
        if dtype_name == 'object':
            if df[col].nunique() / df[col].shape[0] < 0.5:
                df[col] = df[col].astype('category')
        
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
            
        elif dtype_name.startswith('int'):
            df[col] = pd.to_numeric(df[col], downcast='integer')
            
        elif dtype_name.startswith('float'):
            if (df[col].round() == df[col]).all():
                 df[col] = pd.to_numeric(df[col], downcast='integer')
            else:
                 df[col] = pd.to_numeric(df[col], downcast='float')
                 
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f"메모리 사용량: {start_mem:.2f} MB -> {end_mem:.2f} MB")
        print(f"감소율: {(start_mem - end_mem) / start_mem * 100:.1f}%")
        
    return df

In [3]:
train_data = pd.read_csv("data/output/rawdata/train.csv")
train_data['날짜'] = pd.to_datetime(train_data['날짜'])
train_data = downcast(train_data)

test_data = pd.read_csv("data/output/rawdata/test.csv")
test_data = downcast(test_data)

메모리 사용량: 223.99 MB -> 76.27 MB
감소율: 65.9%
메모리 사용량: 13.07 MB -> 2.82 MB
감소율: 78.4%
메모리 사용량: 13.07 MB -> 2.82 MB
감소율: 78.4%


### 중복제거

In [4]:
def handle_duplicates(df):
    """
    데이터프레임의 중복 행을 찾아 출력하고, 첫 번째 행만 남기고 제거합니다.
    
    Args:
        df (pd.DataFrame): 처리할 데이터프레임
        
    Returns:
        pd.DataFrame: 중복이 제거된 데이터프레임
    """
    original_rows = len(df)
    duplicates = df[df.duplicated(keep=False)]

    if not duplicates.empty:
        print(f"총 **{len(duplicates):,}개**({duplicates.duplicated().sum():,}쌍)의 중복 행이 발견되었습니다.")
        print("\n**[중복 데이터 목록]**")
        print("```")
        print(duplicates.sort_values(by=list(df.columns)).to_string())
        print("```")
        
        print("\n중복된 행 중 첫 번째 행만 남기고 모두 제거합니다...")
        processed_df = df.drop_duplicates(keep='first')
        new_rows = len(processed_df)
        print(f"✅ **제거 완료.** 전체 행 수가 `{original_rows:,}`개에서 `{new_rows:,}`개로 변경되었습니다.")
        
        return processed_df
        
    else:
        print("✅ 중복된 행이 없습니다.")
        return df

In [5]:
train_data = handle_duplicates(train_data)

총 **12개**(6쌍)의 중복 행이 발견되었습니다.

**[중복 데이터 목록]**
```
                날짜  월ID  상점ID   상품ID    판매가  판매량                     상점명                                              상품명  상품분류ID            상품분류명
76961   2013-01-05    0    54  20130  149.0    1         Химки ТЦ "Мега"                          УЧЕНИК ЧАРОДЕЯ (регион)      40       Кино - DVD
76962   2013-01-05    0    54  20130  149.0    1         Химки ТЦ "Мега"                          УЧЕНИК ЧАРОДЕЯ (регион)      40       Кино - DVD
1435365 2014-02-23   13    50   3423  999.0    1      Тюмень ТЦ "Гудвин"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1435367 2014-02-23   13    50   3423  999.0    1      Тюмень ТЦ "Гудвин"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1496765 2014-03-23   14    21   3423  999.0    1  Москва МТРЦ "Афи Молл"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1496766 2014-03-23   14    21   3423  999.0    1  Москва МТРЦ "Афи 

### 이상치 제거 (판매가 6000이상, 판매량 5이상)

In [6]:
# 이상치 조건에 해당하는 데이터 확인
outliers = train_data[(train_data['판매가'] >= 6000) | (train_data['판매량'] >= 5)]

print(f"이상치 데이터 수: {len(outliers):,}개")
print("\n[이상치 데이터 샘플 (처음 10개)]")
print(outliers.head(10))

이상치 데이터 수: 66,091개

[이상치 데이터 샘플 (처음 10개)]
             날짜  월ID  상점ID  상품ID      판매가  판매량                  상점명  \
252  2013-01-02    0    25  3432    599.0    5  Москва ТРК "Атриум"   
352  2013-01-15    0    25  2973   2499.0   13  Москва ТРК "Атриум"   
353  2013-01-16    0    25  2973   2499.0    5  Москва ТРК "Атриум"   
404  2013-01-25    0    25  2972    599.0   13  Москва ТРК "Атриум"   
405  2013-01-26    0    25  2972    599.0    5  Москва ТРК "Атриум"   
421  2013-01-26    0    25  3186    419.3    5  Москва ТРК "Атриум"   
1066 2013-01-18    0    25  4861   8490.0    1  Москва ТРК "Атриум"   
1203 2013-01-20    0    25  5613   6190.0    1  Москва ТРК "Атриум"   
1617 2013-01-03    0    25  3321   1999.0    5  Москва ТРК "Атриум"   
1701 2013-01-03    0    25  4384  13499.0    1  Москва ТРК "Атриум"   

                                                    상품명  상품분류ID  \
252               Far Cry 3 [PC, Jewel, русская версия]      30   
352           DmC Devil May Cry [PS3, русс

In [7]:
# 이상치 제거
original_rows = len(train_data)
train_data = train_data[~((train_data['판매가'] >= 6000) | (train_data['판매량'] >= 5))]

# 처리 결과 출력
print(f"이상치 제거 완료:")
print(f"- 전체 행 수: {original_rows:,}개 → {len(train_data):,}개")
print(f"- 제거된 행 수: {original_rows - len(train_data):,}개")
print(f"- 제거 비율: {(original_rows - len(train_data))/original_rows*100:.2f}%")

# 처리 후 통계 확인
print("\n[처리 후 판매가/판매량 통계]")
print(train_data[['판매가', '판매량']].describe([.01, .05, .25, .5, .75, .95, .99]))

이상치 제거 완료:
- 전체 행 수: 2,935,843개 → 2,869,752개
- 제거된 행 수: 66,091개
- 제거 비율: 2.25%

[처리 후 판매가/판매량 통계]
                판매가           판매량
count  2.869752e+06  2.869752e+06
mean   7.524252e+02  1.114773e+00
std    8.088992e+02  4.364864e-01
min   -1.000000e+00 -2.200000e+01
1%     2.800000e+01  1.000000e+00
5%     9.934000e+01  1.000000e+00
25%    2.490000e+02  1.000000e+00
50%    3.990000e+02  1.000000e+00
75%    9.900000e+02  1.000000e+00
95%    2.599000e+03  2.000000e+00
99%    3.598000e+03  3.000000e+00
max    5.999500e+03  4.000000e+00


### 상점명 유사도 비교 후 처리

In [8]:
train_data["상점ID"].unique()

array([59, 25, 24, 23, 19, 22, 18, 21, 28, 27, 29, 26,  4,  6,  2,  3,  7,
        0,  1, 16, 15,  8, 10, 14, 13, 12, 53, 31, 30, 32, 35, 56, 54, 47,
       50, 42, 43, 52, 51, 41, 38, 44, 37, 46, 45,  5, 57, 58, 55, 17,  9,
       49, 39, 40, 48, 34, 33, 20, 11, 36], dtype=int8)

In [9]:
from difflib import SequenceMatcher

def calculate_similarity(str1, str2):
    """두 문자열 간의 유사도를 계산"""
    return SequenceMatcher(None, str1, str2).ratio()

def find_similar_shop_names(df, threshold=0.8):
    """
    상점명 간의 유사도를 계산하고 유사한 쌍을 찾아냅니다.
    
    Args:
        df: 데이터프레임
        threshold: 유사도 임계값 (0~1 사이, 1에 가까울수록 더 유사함)
    """
    shop_names = df['상점명'].unique()
    similar_pairs = []
    
    for i in range(len(shop_names)):
        for j in range(i + 1, len(shop_names)):
            similarity = calculate_similarity(shop_names[i], shop_names[j])
            if similarity >= threshold:
                similar_pairs.append({
                    '상점명1': shop_names[i],
                    '상점명2': shop_names[j],
                    '유사도': similarity
                })
    
    return pd.DataFrame(similar_pairs).sort_values('유사도', ascending=False)

In [10]:
# 유사한 상점명 쌍 찾기 (유사도 70% 이상)
similar_shops = find_similar_shop_names(train_data, threshold=0.7)

if not similar_shops.empty:
    print("유사한 상점명 쌍 발견:")
    print(similar_shops.to_string(index=False))
else:
    print("유사도 70% 이상인 상점명 쌍이 없습니다.")

유사한 상점명 쌍 발견:
                                 상점명1                                            상점명2      유사도
           Жуковский ул. Чкалова 39м?                      Жуковский ул. Чкалова 39м² 0.961538
     Москва ТК "Буденовский" (пав.К7)                Москва ТК "Буденовский" (пав.А2) 0.937500
        !Якутск ТЦ "Центральный" фран                         Якутск ТЦ "Центральный" 0.884615
        !Якутск Орджоникидзе, 56 фран                         Якутск Орджоникидзе, 56 0.884615
РостовНаДону ТРК "Мегацентр Горизонт" РостовНаДону ТРК "Мегацентр Горизонт" Островной 0.880952
              Москва ТЦ "Семеновский"                          Москва ТЦ "Перловский" 0.844444
                      Химки ТЦ "Мега"                                  Омск ТЦ "Мега" 0.827586
      Москва ТЦ "МЕГА Теплый Стан" II                  Москва ТЦ "МЕГА Белая Дача II" 0.754098
                 Уфа ТК "Центральный"                         Якутск ТЦ "Центральный" 0.744186
              Казань ТЦ "ПаркХаус" I

In [11]:
# 완전히 동일한 상점들의 ID 매핑 확인
print("=== 동일 상점 ID 확인 ===")
for shop_pair in [
    ('Жуковский ул. Чкалова 39м?', 'Жуковский ул. Чкалова 39м²'),
    ('!Якутск ТЦ "Центральный" фран', 'Якутск ТЦ "Центральный"'),
    ('!Якутск Орджоникидзе, 56 фран', 'Якутск Орджоникидзе, 56')
]:
    shop1_id = train_data[train_data['상점명'] == shop_pair[0]]['상점ID'].iloc[0]
    shop2_id = train_data[train_data['상점명'] == shop_pair[1]]['상점ID'].iloc[0]
    print(f"\n{shop_pair[0]} (ID: {shop1_id})")
    print(f"{shop_pair[1]} (ID: {shop2_id})")

=== 동일 상점 ID 확인 ===

Жуковский ул. Чкалова 39м? (ID: 10)
Жуковский ул. Чкалова 39м² (ID: 11)

!Якутск ТЦ "Центральный" фран (ID: 1)
Якутск ТЦ "Центральный" (ID: 58)

!Якутск Орджоникидзе, 56 фран (ID: 0)
Якутск Орджоникидзе, 56 (ID: 57)


In [12]:
def unify_shop_ids(df):
    """
    동일 상점에 대해 상점 ID 및 상점명을 통일합니다.
    
    Args:
        df (pd.DataFrame): 상점 데이터가 포함된 데이터프레임
        
    Returns:
        pd.DataFrame: 상점 ID가 통일된 데이터프레임
    """
    id_mapping = {
        0: 57,   # Жуковский ул. Чкалова 39м? -> Жуковский ул. Чкалова 39м²
        1: 58,   # !Якутск ТЦ "Центральный" фран -> Якутск ТЦ "Центральный"
        2: 59    # !Якутск Орджоникидзе, 56 фран -> Якутск Орджоникидзе, 56
    }
    
    for old_id, new_id in id_mapping.items():
        df.loc[df['상점ID'] == old_id, '상점ID'] = new_id
        new_name = df.loc[df['상점ID'] == new_id, '상점명'].iloc[0]
        df.loc[df['상점ID'] == new_id, '상점명'] = new_name

    
    return df

In [13]:
train_data = unify_shop_ids(train_data)
test_data = unify_shop_ids(test_data)   

In [14]:
np.sort(train_data["상점ID"].unique())

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59], dtype=int8)

### 월별 데이터 리셈플링 + 전치를 위한 train, test 병합

In [15]:
# 1) 그룹 키
group_cols = ['월ID', '상점ID', '상품ID']

# 2) 집계
monthly_data = (
    train_data
      .groupby(group_cols)
      .agg(
          월별_판매량=('판매량', 'sum'),
          월별_판매건수=('판매량', 'size'),      # 거래 건수(행 수). 
          월별_평균_판매가=('판매가', 'mean'),
          상점명=('상점명', 'first'),
          상품명=('상품명', 'first'),
          상품분류ID=('상품분류ID', 'first'),
          상품분류명=('상품분류명', 'first')
      )
      .reset_index()
      .rename(columns={
          '월별_판매량': '월별 판매량',
          '월별_판매건수': '월별 판매건수',
          '월별_평균_판매가': '월별 평균 판매가'
      })
)

print("✅ 월별 데이터 집계 + 리네이밍 완료")
monthly_data.head(30)


✅ 월별 데이터 집계 + 리네이밍 완료


,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상점명,상품명,상품분류ID,상품분류명
0,0,3,32,3,3,349.0,"Балашиха ТРК ""Октябрь-Киномир""",1+1,40,Кино - DVD
1,0,3,98,5,5,399.0,"Балашиха ТРК ""Октябрь-Киномир""",1812: УЛАНСКАЯ БАЛЛАДА,40,Кино - DVD
2,0,3,1037,1,1,3500.0,"Балашиха ТРК ""Октябрь-Киномир""",3D GARDEN (Наш Сад 10) (box),75,Программы - Для дома и офиса
3,0,3,1143,1,1,399.0,"Балашиха ТРК ""Октябрь-Киномир""",ABBA The Definitive Collection 2CD,55,Музыка - CD локального производства
4,0,3,1201,1,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Back In Black,55,Музыка - CD локального производства
5,0,3,1204,2,2,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Black Ice,55,Музыка - CD локального производства
6,0,3,1217,2,2,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Iron Man 2,55,Музыка - CD локального производства
7,0,3,1222,1,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Live,55,Музыка - CD локального производства
8,0,3,1224,1,1,399.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Live At River Plate 2CD,55,Музыка - CD локального производства
9,0,3,1235,1,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC The Rasors Edge,55,Музыка - CD локального производства


In [16]:
test_data.head()

,ID,상점ID,상품ID,상점명,상품명,상품분류ID,상품분류명,월ID
0,0,5,5037,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",19,Игры - PS3,34
1,1,5,5320,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,34
2,2,5,5233,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,34
3,3,5,5232,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,34
4,4,5,5268,"Вологда ТРЦ ""Мармелад""","Need for Speed [PS4, русская версия]",20,Игры - PS4,34


In [17]:
# test에 없는 컬럼을 0으로 추가하고, 순서도 train과 동일하게
test_aligned = test_data.reindex(columns=monthly_data.columns, fill_value=0)

# 행 방향으로 결합
full_data = pd.concat([monthly_data, test_aligned], ignore_index=True)

full_data


,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상점명,상품명,상품분류ID,상품분류명
0,0,3,32,3,3,349.0,"Балашиха ТРК ""Октябрь-Киномир""",1+1,40,Кино - DVD
1,0,3,98,5,5,399.0,"Балашиха ТРК ""Октябрь-Киномир""",1812: УЛАНСКАЯ БАЛЛАДА,40,Кино - DVD
2,0,3,1037,1,1,3500.0,"Балашиха ТРК ""Октябрь-Киномир""",3D GARDEN (Наш Сад 10) (box),75,Программы - Для дома и офиса
3,0,3,1143,1,1,399.0,"Балашиха ТРК ""Октябрь-Киномир""",ABBA The Definitive Collection 2CD,55,Музыка - CD локального производства
4,0,3,1201,1,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Back In Black,55,Музыка - CD локального производства
...,...,...,...,...,...,...,...,...,...,...
1800367,34,45,18454,0,0,0.0,"Самара ТЦ ""ПаркХаус""",СБ. Союз 55,55,Музыка - CD локального производства
1800368,34,45,16188,0,0,0.0,"Самара ТЦ ""ПаркХаус""",Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры
1800369,34,45,15757,0,0,0.0,"Самара ТЦ ""ПаркХаус""",НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства
1800370,34,45,19648,0,0,0.0,"Самара ТЦ ""ПаркХаус""",ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD


### 상점 명을 통한 파생 변수

In [18]:

def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    # 숫자형 다운캐스트
    for c in df.select_dtypes(include=['int', 'float']).columns:
        if pd.api.types.is_integer_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], downcast='integer')
        else:
            df[c] = pd.to_numeric(df[c], downcast='float')

    # ID/월 컬럼이 있으면 부호 없는 정수로(음수 없다고 가정 시)
    for c in ['월ID', '상점ID', '상품ID']:
        if c in df.columns and pd.api.types.is_numeric_dtype(df[c]):
            mn, mx = df[c].min(), df[c].max()
            if pd.notna(mn) and mn >= 0:
                if mx <= 255:   df[c] = df[c].astype('uint8')
                elif mx <= 65535: df[c] = df[c].astype('uint16')
                elif mx <= 4294967295: df[c] = df[c].astype('uint32')

    # 자주 쓰는 문자열은 category로
    for c in ['상점명', '상품분류명', '상품명']:
        if c in df.columns and df[c].dtype == 'object':
            df[c] = df[c].astype('category')
    return df

def add_shop_features_vectorized(df: pd.DataFrame) -> pd.DataFrame:
    # 문자열 처리용 뷰 (카테고리여도 string으로 일시 변환해 연산)
    name = df['상점명'].astype('string')
    name_clean = name.str.replace(r'^\!+', '', regex=True)

    # channel (벡터화)
    m_online = name.str.contains('Интернет-магазин', case=False, na=False)
    m_wh = name.str.contains('1С-Онлайн|Цифровой склад', case=False, na=False)
    m_field = name.str.contains('Выездная Торговля', case=False, na=False)
    channel = np.select(
        [m_online, m_wh, m_field],
        ['online', 'warehouse_online', 'field'],
        default='offline'
    )

    # city (따옴표/몰유형/주소 전까지 캡처)
    pat_city = r'^([А-ЯA-ZЁ][^"«(]*?)\s*(?=(МТРЦ|ТРК|ТРЦ|ТЦ|ТК|"|«|\(|ул\.|Магазин|Интернет|Цифровой|Выездная|$))'
    city = name_clean.str.extract(pat_city, expand=False)[0]
    city = city.mask(m_online | m_wh | m_field)  # 특수 채널은 결측 처리

    # mall_type
    mall_type = name_clean.str.extract(r'\b(МТРЦ|ТРК|ТРЦ|ТЦ|ТК)\b', expand=False).str.upper()

    # 메모리 효율적으로 직접 할당 + 카테고리화
    df.loc[:, '도시'] = city.astype('category')
    df.loc[:, '쇼핑몰 종류'] = mall_type.astype('category')
    df.loc[:, '채널'] = pd.Categorical(channel, categories=['offline','online','warehouse_online','field'])

    return df


full_data = optimize_dtypes(full_data)
full_data = add_shop_features_vectorized(full_data)

full_data


,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상점명,상품명,상품분류ID,상품분류명,도시,쇼핑몰 종류,채널
0,0,3,32,3,3,349.0,"Балашиха ТРК ""Октябрь-Киномир""",1+1,40,Кино - DVD,Балашиха,ТРК,offline
1,0,3,98,5,5,399.0,"Балашиха ТРК ""Октябрь-Киномир""",1812: УЛАНСКАЯ БАЛЛАДА,40,Кино - DVD,Балашиха,ТРК,offline
2,0,3,1037,1,1,3500.0,"Балашиха ТРК ""Октябрь-Киномир""",3D GARDEN (Наш Сад 10) (box),75,Программы - Для дома и офиса,Балашиха,ТРК,offline
3,0,3,1143,1,1,399.0,"Балашиха ТРК ""Октябрь-Киномир""",ABBA The Definitive Collection 2CD,55,Музыка - CD локального производства,Балашиха,ТРК,offline
4,0,3,1201,1,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Back In Black,55,Музыка - CD локального производства,Балашиха,ТРК,offline
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800367,34,45,18454,0,0,0.0,"Самара ТЦ ""ПаркХаус""",СБ. Союз 55,55,Музыка - CD локального производства,Самара,ТЦ,offline
1800368,34,45,16188,0,0,0.0,"Самара ТЦ ""ПаркХаус""",Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,Самара,ТЦ,offline
1800369,34,45,15757,0,0,0.0,"Самара ТЦ ""ПаркХаус""",НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,Самара,ТЦ,offline
1800370,34,45,19648,0,0,0.0,"Самара ТЦ ""ПаркХаус""",ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,Самара,ТЦ,offline


### 상품 분류명을 통한 파생 변수

In [19]:
full_data[["상품대분류", "상품소분류"]] = full_data["상품분류명"].str.split(" - ", n=1, expand=True)

In [20]:
full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상점명,상품명,상품분류ID,상품분류명,도시,쇼핑몰 종류,채널,상품대분류,상품소분류
0,0,3,32,3,3,349.0,"Балашиха ТРК ""Октябрь-Киномир""",1+1,40,Кино - DVD,Балашиха,ТРК,offline,Кино,DVD
1,0,3,98,5,5,399.0,"Балашиха ТРК ""Октябрь-Киномир""",1812: УЛАНСКАЯ БАЛЛАДА,40,Кино - DVD,Балашиха,ТРК,offline,Кино,DVD
2,0,3,1037,1,1,3500.0,"Балашиха ТРК ""Октябрь-Киномир""",3D GARDEN (Наш Сад 10) (box),75,Программы - Для дома и офиса,Балашиха,ТРК,offline,Программы,Для дома и офиса
3,0,3,1143,1,1,399.0,"Балашиха ТРК ""Октябрь-Киномир""",ABBA The Definitive Collection 2CD,55,Музыка - CD локального производства,Балашиха,ТРК,offline,Музыка,CD локального производства
4,0,3,1201,1,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Back In Black,55,Музыка - CD локального производства,Балашиха,ТРК,offline,Музыка,CD локального производства


In [21]:
full_data["상품소분류"].nunique()

60

### 필요없는 명칭 열 제거

In [22]:
full_data.drop(columns=['상점명','상품분류명', '상품명'], inplace=True)
full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,도시,쇼핑몰 종류,채널,상품대분류,상품소분류
0,0,3,32,3,3,349.0,40,Балашиха,ТРК,offline,Кино,DVD
1,0,3,98,5,5,399.0,40,Балашиха,ТРК,offline,Кино,DVD
2,0,3,1037,1,1,3500.0,75,Балашиха,ТРК,offline,Программы,Для дома и офиса
3,0,3,1143,1,1,399.0,55,Балашиха,ТРК,offline,Музыка,CD локального производства
4,0,3,1201,1,1,299.0,55,Балашиха,ТРК,offline,Музыка,CD локального производства


### 신상 여부 열 추가

In [23]:
# 1) 시간열 → 월 인덱스(연*12+월)
def _to_month_index(s: pd.Series) -> pd.Series:
    if is_datetime64_any_dtype(s):
        sd = pd.to_datetime(s, errors='coerce')
        return (sd.dt.year * 12 + sd.dt.month).astype('Int32')
    if isinstance(s.dtype, pd.PeriodDtype):
        sp = s.astype('period[M]')
        return (sp.year * 12 + sp.month).astype('Int32')
    return pd.to_numeric(s, errors='coerce').astype('Int32')
    return df

# 3) 첫판매월/경과개월/신상여부(0/1) — 누적 최소(cummin) 기반, merge 없음
def add_first_sale_features_causal(
    df: pd.DataFrame,
    time_col: str,
    specs: List[Dict[str, Union[str, List[str]]]],
    int_dtype: str = 'Int16',
    add_ts: bool = False  # 필요 시 True로 바꾸면 월초 timestamp 컬럼도 추가
) -> pd.DataFrame:
    mi = _to_month_index(df[time_col])

    for spec in specs:
        keys = list(spec['keys'])
        name = str(spec['name'])

        first_idx_col = f'{name}_첫판매_월ID'
        age_col       = f'{name}_경과개월'
        new_col       = f'{name}_신상여부'
        ts_col        = f'{name}_첫판매월'

        # 재실행 안전: 기존 동일 열 제거
        drop_cols = [c for c in [first_idx_col, age_col, new_col, ts_col] if c in df.columns]
        if drop_cols:
            df.drop(columns=drop_cols, inplace=True)

        # 정렬용 뷰 만들기
        tmp = pd.DataFrame({'__mi__': mi}, index=df.index)
        for k in keys:
            tmp[k] = df[k].values

        # 키+시간 정렬 → 그룹별 누적 최소(첫 관측월 유지)
        tmp_sorted = tmp.sort_values(by=keys + ['__mi__'], kind='mergesort')
        first_sorted = tmp_sorted.groupby(keys, sort=False)['__mi__'].cummin()

        # 원래 순서로 복원
        first_idx = pd.Series(pd.NA, index=df.index, dtype='Int64')
        first_idx.loc[tmp_sorted.index] = first_sorted.values

        # 파생 열 생성
        df[first_idx_col] = first_idx.astype(int_dtype)
        df[age_col] = (mi - first_idx).astype(int_dtype)

        # 신상여부: 0/1(uint8), 결측은 0
        df[new_col] = ((df[age_col] == 0) & df[age_col].notna()).astype('uint8')

        # 옵션: 읽기 쉬운 첫판매월(월초 timestamp)
        if add_ts and (is_datetime64_any_dtype(df[time_col]) or isinstance(df[time_col].dtype, pd.PeriodDtype)):
            # month index → 월초 timestamp
            y = ((first_idx - 1) // 12).astype('Int32')
            m = (first_idx - y * 12).astype('Int8')
            df[ts_col] = pd.to_datetime(pd.DataFrame({'year': y, 'month': m, 'day': 1}), errors='coerce')

    return df



In [24]:
# 2) 첫판매/경과/신상 (예: 상품, 상품×상점 기준)
specs = [
    {'keys': ['상품ID'],              'name': '상품'},
    {'keys': ['상점ID','상품ID'],     'name': '상품_상점'}
]
# time_col이 정수형 '월ID'든 datetime 열이든 그대로 넣으면 됩니다.
full_data = add_first_sale_features_causal(full_data, time_col='월ID', specs=specs, int_dtype='Int16', add_ts=False)

full_data[full_data["상품_첫판매_월ID"] == 32]

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,도시,쇼핑몰 종류,채널,상품대분류,상품소분류,상품_첫판매_월ID,상품_경과개월,상품_신상여부,상품_상점_첫판매_월ID,상품_상점_경과개월,상품_상점_신상여부
1526231,32,3,2791,1,1,5499.000000,20,Балашиха,ТРК,offline,Игры,PS4,32,0,1,32,0,1
1526244,32,3,3350,7,5,2849.000000,19,Балашиха,ТРК,offline,Игры,PS3,32,0,1,32,0,1
1526245,32,3,3351,4,3,3915.666748,20,Балашиха,ТРК,offline,Игры,PS4,32,0,1,32,0,1
1526246,32,3,3352,7,3,2999.000000,23,Балашиха,ТРК,offline,Игры,XBOX 360,32,0,1,32,0,1
1526247,32,3,3353,1,1,3999.000000,24,Балашиха,ТРК,offline,Игры,XBOX ONE,32,0,1,32,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1799955,34,45,2379,0,0,0.000000,31,Самара,ТЦ,offline,Игры PC,Цифра,32,2,0,34,0,1
1800028,34,45,18188,0,0,0.000000,57,Самара,ТЦ,offline,Музыка,MP3,32,2,0,34,0,1
1800033,34,45,4842,0,0,0.000000,24,Самара,ТЦ,offline,Игры,XBOX ONE,32,2,0,32,2,0
1800254,34,45,2664,0,0,0.000000,58,Самара,ТЦ,offline,Музыка,Винил,32,2,0,34,0,1


In [25]:
full_data.drop(columns=['상품_첫판매_월ID', '상품_상점_첫판매_월ID'], inplace=True)
full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,도시,쇼핑몰 종류,채널,상품대분류,상품소분류,상품_경과개월,상품_신상여부,상품_상점_경과개월,상품_상점_신상여부
0,0,3,32,3,3,349.0,40,Балашиха,ТРК,offline,Кино,DVD,0,1,0,1
1,0,3,98,5,5,399.0,40,Балашиха,ТРК,offline,Кино,DVD,0,1,0,1
2,0,3,1037,1,1,3500.0,75,Балашиха,ТРК,offline,Программы,Для дома и офиса,0,1,0,1
3,0,3,1143,1,1,399.0,55,Балашиха,ТРК,offline,Музыка,CD локального производства,0,1,0,1
4,0,3,1201,1,1,299.0,55,Балашиха,ТРК,offline,Музыка,CD локального производства,0,1,0,1


### 년, 월, 분기 열 생성

In [26]:
mi = pd.to_numeric(full_data['월ID'], errors='coerce').astype('Int16')

full_data['연']   = (2013 + (mi // 12)).astype('Int16')
full_data['월']   = (1 + (mi % 12)).astype('Int8')
full_data['분기'] = ((full_data['월'] - 1) // 3 + 1).astype('Int8')

full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,도시,쇼핑몰 종류,채널,상품대분류,상품소분류,상품_경과개월,상품_신상여부,상품_상점_경과개월,상품_상점_신상여부,연,월,분기
0,0,3,32,3,3,349.0,40,Балашиха,ТРК,offline,Кино,DVD,0,1,0,1,2013,1,1
1,0,3,98,5,5,399.0,40,Балашиха,ТРК,offline,Кино,DVD,0,1,0,1,2013,1,1
2,0,3,1037,1,1,3500.0,75,Балашиха,ТРК,offline,Программы,Для дома и офиса,0,1,0,1,2013,1,1
3,0,3,1143,1,1,399.0,55,Балашиха,ТРК,offline,Музыка,CD локального производства,0,1,0,1,2013,1,1
4,0,3,1201,1,1,299.0,55,Балашиха,ТРК,offline,Музыка,CD локального производства,0,1,0,1,2013,1,1
